In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import Adam
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
import os
import shutil
import pandas as pd

## Model Construction

In [3]:
model = Sequential()
model.add(Conv2D(32,(3,3),activation = "relu",input_shape=(224,224,3)))
model.add(Conv2D(64,(3,3),activation = "relu"))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(64,(3,3),activation = 'relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128,(3,3),activation ='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(64,activation = "relu"))
model.add(Dropout(0.4))
model.add(Dense(1,activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy',optimizer = "adam",metrics = ["accuracy"])

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 110, 110, 64)      0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 54, 54, 64)        0         
__________

## Preparation of generators

In [5]:
train_gen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

val_gen = image.ImageDataGenerator(
    rescale = 1./255
)

In [6]:
train_generator = train_gen.flow_from_directory(
    "covid19dataset/train",
    target_size = (224,224),
    batch_size = 32,
    class_mode = "binary",
)
val_generator = val_gen.flow_from_directory(
    "covid19dataset/val",
    target_size = (224,224),
    batch_size = 32,
    class_mode = "binary",
)

Found 234 images belonging to 2 classes.
Found 56 images belonging to 2 classes.


In [7]:
train_generator.class_indices

{'Covid19': 0, 'Normal': 1}

## Model Training

In [8]:
from keras.callbacks import ModelCheckpoint,EarlyStopping

In [9]:
checkpoint = ModelCheckpoint("./model.hdf5",monitor = 'val_acc',save_best_only = True,mode = 'max')
#earlyStop = EarlyStopping(monitor = 'val_acc',patience = 4,min_delta = 0.01,mode = 'max')

In [10]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=9,
    epochs=10,
    validation_data = val_generator,
    validation_steps = 2,
    shuffle=True,
    callbacks = [checkpoint]
)

Epoch 1/10
9/9 [==============================] - 59s 7s/step - loss: 2.0970 - acc: 0.4795 - val_loss: 0.6914 - val_acc: 0.5000
Epoch 2/10
9/9 [==============================] - 55s 6s/step - loss: 0.6873 - acc: 0.5246 - val_loss: 0.6840 - val_acc: 0.8036
Epoch 3/10
9/9 [==============================] - 51s 6s/step - loss: 0.6500 - acc: 0.6081 - val_loss: 0.5796 - val_acc: 0.9643
Epoch 4/10
9/9 [==============================] - 51s 6s/step - loss: 0.4976 - acc: 0.7798 - val_loss: 0.2884 - val_acc: 0.9643
Epoch 5/10
9/9 [==============================] - 50s 6s/step - loss: 0.3765 - acc: 0.8356 - val_loss: 0.2642 - val_acc: 0.9821
Epoch 6/10
9/9 [==============================] - 59s 7s/step - loss: 0.3032 - acc: 0.8950 - val_loss: 0.1513 - val_acc: 0.9464
Epoch 7/10
9/9 [==============================] - 48s 5s/step - loss: 0.3249 - acc: 0.8671 - val_loss: 0.1497 - val_acc: 0.9821
Epoch 8/10
9/9 [==============================] - 44s 5s/step - loss: 0.2350 - acc: 0.9012 - val_loss: 0

In [11]:
# To load the model use : 
# model = keras.models.load_model("./model.hdf5")